In [1]:
from data.BikeDataImporter import BikeDataImporter
from data.DataPreparing import DataPreparing

from models.LightGBM_Model import LightGBM_Model
from models.MSTL_Model import MSTL_Model
from models.LinearQuantileRegression_Model import LinearQuantileRegression_Model
from models.GradientBoostingRegressor_Model import GradientBoostingRegressor_Model
from models.Baseline_Model import Baseline_Model

import optuna
from sklearn.metrics import mean_pinball_loss
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import d2_pinball_score
from statsmodels.tsa.seasonal import MSTL
from matplotlib import pyplot as plt
import os
from pathlib import Path
from sklearn.linear_model import QuantileRegressor
from sklearn.impute import SimpleImputer
from sklearn.ensemble import GradientBoostingRegressor
from statsforecast import StatsForecast
from statsforecast.models import MSTL
from sklearn.preprocessing import LabelEncoder
from typing import Tuple, Dict, List
import seaborn as sns

c:\Users\simon\.conda\envs\ptsc\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_preparer = DataPreparing()
data = data_preparer.get_bike_and_weather_data()

# Select either the iterative or the non-iterative feature creation
# iterative = False
# if iterative:
#     print("iterative")
#     data = data_preparer.create_features_iterative(data)
# else:
#     print("non-iterative")
#     data = data_preparer.create_features(data)

FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\simon\\OneDrive\\Dokumente\\[1] Uni\\[1] Master\\3. Semester Wintersemester 2024\\PTSC\\Code_PTSC\\Probabilistic_time_series_challenge-\\data\\weather\\history\\karlsruhe\\history_karlsruhe_20250104.csv'

In [ ]:
# fmt: off
# removed 'snow_depth' from Features_iterative
FEATURES_ITERATIVE = [
    'temperature_2m', 'relative_humidity_2m', 'dew_point_2m',
       'apparent_temperature', 'precipitation', 'rain', 'snowfall',
       'pressure_msl', 'surface_pressure', 'cloud_cover',
       'cloud_cover_low', 'cloud_cover_mid', 'cloud_cover_high',
       'et0_fao_evapotranspiration', 'vapour_pressure_deficit',
       'wind_speed_10m', 'wind_speed_100m', 'wind_direction_10m',
       'wind_direction_100m', 'wind_gusts_10m', 'soil_temperature_0_to_7cm',
       'soil_moisture_0_to_7cm', 'wet_bulb_temperature_2m',
       'sunshine_duration', 'shortwave_radiation', 'direct_radiation',
       'diffuse_radiation', 'direct_normal_irradiance',
       'global_tilted_irradiance', 'terrestrial_radiation',
       'shortwave_radiation_instant', 'direct_radiation_instant',
       'diffuse_radiation_instant', 'direct_normal_irradiance_instant',
       'global_tilted_irradiance_instant', 'terrestrial_radiation_instant',
       'month', 'day', 'quarter', 'weekday', 'lag_1', 'lag_2', 'lag_3',
       'lag_4', 'lag_5', 'lag_6', 'lag_7', 'lag_14', 'lag_20', 'lag_28',
       'lag_50', 'diff_lag_1', 'diff_lag_2', 'diff_lag_3', 'diff_lag_4',
       'is_holiday', 'corona_phase', 'rolling_mean_7', 'rolling_mean_30',
       'rolling_std_7', 'rolling_std_30', 'diff_prev_day']

FEATURES = ['temperature_2m', 'relative_humidity_2m', 'dew_point_2m',
       'apparent_temperature', 'precipitation', 'rain', 'snowfall',
       'snow_depth', 'pressure_msl', 'surface_pressure', 'cloud_cover',
       'cloud_cover_low', 'cloud_cover_mid', 'cloud_cover_high',
       'et0_fao_evapotranspiration', 'vapour_pressure_deficit',
       'wind_speed_10m', 'wind_speed_100m', 'wind_direction_10m',
       'wind_direction_100m', 'wind_gusts_10m', 'soil_temperature_0_to_7cm',
       'soil_moisture_0_to_7cm', 'wet_bulb_temperature_2m',
       'sunshine_duration', 'shortwave_radiation', 'direct_radiation',
       'diffuse_radiation', 'direct_normal_irradiance',
       'global_tilted_irradiance', 'terrestrial_radiation',
       'shortwave_radiation_instant', 'direct_radiation_instant',
       'diffuse_radiation_instant', 'direct_normal_irradiance_instant',
       'global_tilted_irradiance_instant', 'terrestrial_radiation_instant',
       'month', 'day', 'quarter', 'weekday', 'lag_5', 'lag_6', 'lag_7',
       'lag_8', 'is_holiday', 'corona_phase', 'rolling_mean_7',
       'rolling_mean_30', 'rolling_std_7', 'rolling_std_30', 'diff_prev_day']

TARGET = "bike_count"
# fmt: on

In [ ]:
#####################################################
# Hyperparameters for the LightGBM model
#####################################################
learning_rate = {
    0.025: 0.021087808348992185,
    0.25: 0.022167552153993537,
    0.5: 0.059787301961913765,
    0.75: 0.055910736122858566,
    0.975: 0.03448286926599699,
}
n_estimators = {0.025: 492, 0.25: 698, 0.5: 496, 0.75: 414, 0.975: 617}
num_leaves = {0.025: 101, 0.25: 149, 0.5: 113, 0.75: 688, 0.975: 988}
reg_alpha = {
    0.025: 0.6791056871910959,
    0.25: 0.16632101115752465,
    0.5: 0.10100252286347253,
    0.75: 0.17434059178457956,
    0.975: 0.5621214661485234,
}
FEATURES_ITERATIVE = {
    0.025: FEATURES_ITERATIVE,
    0.25: FEATURES_ITERATIVE,
    0.5: FEATURES_ITERATIVE,
    0.75: FEATURES_ITERATIVE,
    0.975: FEATURES_ITERATIVE,
}

#####################################################
# Hyperparameters for the MSTL model
#####################################################
season_length_one_value = 7  # weekly seasonality
season_length_two_value = 365  # yearly seasonality

season_length_one = {
    0.025: season_length_one_value,
    0.25: season_length_one_value,
    0.5: season_length_one_value,
    0.75: season_length_one_value,
    0.975: season_length_one_value,
}
season_length_two = {
    0.025: season_length_two_value,
    0.25: season_length_two_value,
    0.5: season_length_two_value,
    0.75: season_length_two_value,
    0.975: season_length_two_value,
}

#####################################################
# Hyperparameters for the LinearQuantileRegression model
#####################################################
fit_intercept = 1  # True
alpha = {
    0.025: 0.00011033481513836356,
    0.25: 3.733460861903541e-05,
    0.5: 0.00012765214862505564,
    0.75: 0.001978444138077672,
    0.975: 0.00093913020701894,
}
FEATURES_NON_ITERATIVE = {
    0.025: FEATURES,
    0.25: FEATURES,
    0.5: FEATURES,
    0.75: FEATURES,
    0.975: FEATURES,
}

In [ ]:
###########################
# Train the lightgbm model
###########################
# lgbm = LightGBM_Model() # TODO Still adjust hyperparameters
# model_lgbm, pred_lgbm, res_lgbm, y_true_dict = lgbm.lightgbm_model_iterative(data, FEATURES_ITERATIVE, TARGET, learning_rate, n_estimators, num_leaves, reg_alpha, [0.025, 0.25, 0.5, 0.75, 0.975])
# Overall Pinnball Loss: 128.89196000299955
# Overall Relative Pinnball Loss: 0.0318052962342797
# Overall D2 Pinball Score: 0.410804769895055

###########################
# Train the MSTL model
###########################
# mstl = MSTL_Model() # TODO Adjust the hyperparameters for season_length_one and season_length_two
# model_mstl, pred_mstl, res_mstl = mstl.mstl_model(data, 700, season_length_one, season_length_two, quantiles = [0.025, 0.25, 0.5, 0.75, 0.975])
# Overall Pinball Loss: 253.12784969163005
# Overall Relative Pinball Loss: 0.11553771490454538
# Overall D2 Pinball Score: -4.958230867574192

############################################
# Train the LinearQuantileRegression model
############################################
# lqr = LinearQuantileRegression_Model()
# model_lqr, pred_lqr, res_lqr = lqr.quantile_regression_model_iterative(data, FEATURES_ITERATIVE, TARGET, fit_intercept, alpha, [0.025, 0.25, 0.5, 0.75, 0.975])
# Mean pinball loss: 8.26454415967746
# Mean relative pinball loss: 0.0025270253775692337
# Mean d2 pinball score: 0.9450100765308097

###########################################
# Train the GradientBoostingRegressor model
###########################################
# gbr = GradientBoostingRegressor_Model()
# model_gbr, pred_gbr, res_gbr = gbr.gradientboostingregressor_model_iterative(data, FEATURES_ITERATIVE, TARGET, [0.025, 0.25, 0.5, 0.75, 0.975])
# Mean pinball loss: 76.78476140161466
# Mean relative pinball loss: 0.019371678944341976
# Mean d2 pinball score: 0.44442737284859407

############################################
# Train the Baseline model
############################################
# baseline = Baseline_Model()
# pred_baseline, res_baseline = baseline.baseline_model(data, TARGET, [0.025, 0.25, 0.5, 0.75, 0.975], last_observations=100)
# Mean pinball loss: 347.4541666666667
# Mean relative pinball loss: 0.08799900633668664
# Mean d2 pinball score: -0.30794383151021376